In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

load_dotenv()
client = OpenAI() # api_key=

pc = Pinecone() # api_key=
index = pc.Index("ta")

Extract text content from transcript files(.txt).

In [2]:
import os
def txt_text(txt_file):
    text = ""
    with open(txt_file, 'r') as file:
        for line in file.readlines():
            if line != '\n':
                text += line
    return text

folder_path='./transcript'
files=os.listdir(folder_path)
trans_texts={}
for file in files: 
    path=folder_path+'/'+file
    trans_text=txt_text(path)
    trans_texts[file]=trans_text

Split the text into chunks and do the embedding for chunks, lastly store the vectors in Pinecone.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks={}
for file_name, trans_text in trans_texts.items():
    text_chunks[file_name]=text_splitter.split_text(trans_text)

In [4]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
index_name = "ta"
for file_name, text_chunk in text_chunks.items():
    docsearch = PineconeVectorStore.from_texts(text_chunk, embeddings, index_name=index_name, namespace='transcript_data', metadatas=[{'file_name': file_name} for _ in text_chunk])

In [5]:
#index.delete(delete_all=True, namespace='transcript_data')